<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/misc/MLM_Encoder_Decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
# Notes
# 1. Figure out MLM in HuggingFace API - Done
# 2. Figure out the mask token id issue - Done
# 3. Create DataLoader for SQUAD Dataset.

In [ ]:
! pip install -qU transformers datasets sentencepiece

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader

from transformers import T5Tokenizer, T5EncoderModel
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset

In [2]:
dataset = load_dataset("squad")

  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model_name = "t5-small"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

tokenizer = T5Tokenizer.from_pretrained(model_name, model_max_length=512)
tokenizer.add_tokens(["<mask>"])
tokenizer.mask_token = "<mask>"
tokenizer.mask_token_id = tokenizer.convert_tokens_to_ids("<mask>")
print(f"Mask Token ID: {tokenizer.mask_token_id}")
model = T5EncoderModel.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
model.to(device)

params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {params}")

Mask Token ID: 32100


Some weights of the model checkpoint at t5-small were not used when initializing T5EncoderModel: ['decoder.block.5.layer.0.SelfAttention.q.weight', 'decoder.block.5.layer.1.EncDecAttention.k.weight', 'decoder.block.3.layer.1.EncDecAttention.k.weight', 'decoder.block.3.layer.1.layer_norm.weight', 'decoder.block.1.layer.1.EncDecAttention.q.weight', 'decoder.block.4.layer.0.layer_norm.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.5.layer.0.SelfAttention.k.weight', 'decoder.block.2.layer.0.SelfAttention.o.weight', 'decoder.block.1.layer.0.SelfAttention.k.weight', 'decoder.block.1.layer.0.SelfAttention.v.weight', 'decoder.block.4.layer.1.EncDecAttention.o.weight', 'decoder.block.1.layer.2.DenseReluDense.wi.weight', 'decoder.block.2.layer.0.SelfAttention.q.weight', 'decoder.block.2.layer.2.DenseReluDense.wi.weight', 'decoder.block.1.layer.1.EncDecAttention.o.weight', 'decoder.block.4.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.k.wei

Total parameters: 35316992


In [4]:
collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                           mlm=True,
                                           return_tensors="pt")

In [7]:
class CustomDataset(Dataset):
  def __init__(self,):
    pass
  
  def __len__(self,):
    pass
  
  def __getitem__(self, idx):
    pass
  
  def collate_fn(self, batch):
    pass